In [ ]:
#!pip3 install torch

In [1]:
import numpy as np
from scipy.io import loadmat

In [2]:
data = loadmat('prepared_data.mat')
neural_data_tensor = data['neural_data_tensor']  
kinematic_tensor = data['kinematic_tensor']      
mean_spikes = data['mean_spikes'].flatten()      
std_spikes = data['std_spikes'].flatten()        
print(f"Neural data shape: {neural_data_tensor.shape}")
print(f"Kinematic data shape: {kinematic_tensor.shape}")

Neural data shape: (496, 170, 67)
Kinematic data shape: (496, 170, 6)


In [3]:
from sklearn.model_selection import train_test_split
reaches = neural_data_tensor.shape[0]
train_idx, test_idx = train_test_split(np.arange(reaches), test_size=0.2, random_state=42)
train_idx, val_idx = train_test_split(train_idx, test_size=0.25, random_state=42)  # 20% validation
X_train = neural_data_tensor[train_idx]
Y_train = kinematic_tensor[train_idx]
X_val = neural_data_tensor[val_idx]
Y_val = kinematic_tensor[val_idx]
X_test = neural_data_tensor[test_idx]
Y_test = kinematic_tensor[test_idx]
print(f"Train shape: {X_train.shape}, Validation shape: {X_val.shape}, Test shape: {X_test.shape}")


Train shape: (297, 170, 67), Validation shape: (99, 170, 67), Test shape: (100, 170, 67)


In [5]:
import torch
import torch.nn as nn

class NeuralDecoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(NeuralDecoderRNN, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.dropout = nn.Dropout(p=0.2) 
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.dropout(out)
        out = self.fc(out)
        return out

input_size = X_train.shape[2]  
hidden_size = 64               
output_size = Y_train.shape[2] 
num_layers = 1                 

model = NeuralDecoderRNN(input_size, hidden_size, output_size, num_layers)


In [6]:
from torch.utils.data import DataLoader, TensorDataset

X_train_torch = torch.tensor(X_train, dtype=torch.float32)
Y_train_torch = torch.tensor(Y_train, dtype=torch.float32)
X_val_torch = torch.tensor(X_val, dtype=torch.float32)
Y_val_torch = torch.tensor(Y_val, dtype=torch.float32)

batch_size = 32
train_loader = DataLoader(TensorDataset(X_train_torch, Y_train_torch), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val_torch, Y_val_torch), batch_size=batch_size)


In [7]:
# Loss and optimizer
criterion = nn.MSELoss()  
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-3)

num_epochs = 200
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for X_batch, Y_batch in train_loader:
        outputs = model(X_batch)
        loss = criterion(outputs, Y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for X_batch, Y_batch in val_loader:
            outputs = model(X_batch)
            loss = criterion(outputs, Y_batch)
            val_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss/len(train_loader):.4f}, Validation Loss: {val_loss/len(val_loader):.4f}")


Epoch 1/200, Train Loss: 549.7490, Validation Loss: 588.3491
Epoch 2/200, Train Loss: 543.8764, Validation Loss: 587.4729
Epoch 3/200, Train Loss: 550.7702, Validation Loss: 585.7706
Epoch 4/200, Train Loss: 541.1102, Validation Loss: 581.7910
Epoch 5/200, Train Loss: 538.7673, Validation Loss: 576.7584
Epoch 6/200, Train Loss: 545.6488, Validation Loss: 568.8636
Epoch 7/200, Train Loss: 527.6658, Validation Loss: 563.1033
Epoch 8/200, Train Loss: 526.1552, Validation Loss: 554.2939
Epoch 9/200, Train Loss: 515.1100, Validation Loss: 548.4693
Epoch 10/200, Train Loss: 507.2189, Validation Loss: 541.0808
Epoch 11/200, Train Loss: 500.6980, Validation Loss: 535.0841
Epoch 12/200, Train Loss: 489.7675, Validation Loss: 528.8688
Epoch 13/200, Train Loss: 496.4371, Validation Loss: 522.9589
Epoch 14/200, Train Loss: 480.4661, Validation Loss: 516.8959
Epoch 15/200, Train Loss: 475.7865, Validation Loss: 513.7797
Epoch 16/200, Train Loss: 470.4595, Validation Loss: 507.4437
Epoch 17/200, Tra

In [11]:
model.eval()
X_test_torch = torch.tensor(X_test, dtype=torch.float32)
Y_test_torch = torch.tensor(Y_test, dtype=torch.float32)
with torch.no_grad():
    predictions = model(X_test_torch)
    test_loss = criterion(predictions, Y_test_torch)
    print(f"Test Loss: {test_loss.item():.4f}")
from sklearn.metrics import mean_absolute_error, r2_score
preds = predictions.detach().cpu().numpy() * kin_std + kin_mean  
targets = Y_test_torch.detach().cpu().numpy() * kin_std + kin_mean  
mae = mean_absolute_error(targets.flatten(), preds.flatten())
r2 = r2_score(targets.flatten(), preds.flatten())

print(f"Validation MAE: {mae:.4f}, R²: {r2:.4f}")

Test Loss: 381.4996
Validation MAE: 296.3833, R²: 0.3070


In [ ]:
'''
# Save model
torch.save(model.state_dict(), "neural_decoder_rnn.pth")

# Load model
model = NeuralDecoderRNN(input_size, hidden_size, output_size, num_layers)
model.load_state_dict(torch.load("neural_decoder_rnn.pth"))
'''